In [3]:
import numpy as np
import pandas as pd
import os
import warnings
import random
import torch 
from torch import nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
from transformers import AlbertModel, AlbertConfig, AlbertTokenizer
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import RobertaForSequenceClassification, AlbertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from torch.utils.data import Dataset,DataLoader, TensorDataset
from torch.autograd import Variable
from torch.utils.data.sampler import SequentialSampler, RandomSampler, WeightedRandomSampler
import numpy as np
from tqdm.auto import tqdm, trange
import time
import matplotlib.pyplot as plt
import copy

warnings.filterwarnings('ignore')

In [4]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('summary/word_tensorboard_pt')
# model = BertModel.from_pretrained('/home/jupyter/pretrained-models/fin-bert/')
model = RobertaForSequenceClassification.from_pretrained(
    "/home/jupyter/pretrained-models/roberta-base/", 
    num_labels = 3,   
    output_attentions = False, 
    output_hidden_states = False, )
model.load_state_dict(torch.load("/home/jupyter/saved-models/roberta-base-500-finetuned.pth"))
model = model.roberta
tokenizer = RobertaTokenizer.from_pretrained('/home/jupyter/pretrained-models/roberta-base/')

In [7]:
#tokenizer = BertTokenizer.from_pretrained('/home/jupyter/pretrained-models/fin-bert/')
#words = tokenizer.vocab.keys()
#word_embedding = model.embeddings.word_embeddings.weight
#writer.add_embedding(word_embedding,
#                         metadata  = words,
#                        tag = f'word embedding')

In [5]:
#tokenizer = BertTokenizer.from_pretrained('/home/jupyter/pretrained-models/fin-bert/')
words = tokenizer.encoder.keys()
word_embedding = model.embeddings.word_embeddings.weight
writer.add_embedding(word_embedding,
                         metadata  = words,
                        tag = f'word embedding')

In [13]:
%tensorboard --logdir summary

UsageError: Line magic function `%tensorboard` not found.


In [12]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df_reports = pd.read_pickle("reports_processed.pkl")
df_reports.sample(10)

,publishedDate,targetPriceChange,bullets,recommendationId,total_text,prepped_total_text
14582,2010-06-11 06:41:00+00:00,False,[{'text': 'Clas Ohlson reported a Q4 pre-tax p...,2,Below market expectations but better than our ...,below market expectations but better than our ...
19100,2015-06-30 18:17:00+00:00,False,[{'text': 'Following the decline in stock mark...,2,We expect Q2 NAV of SEK 175 per share. Followi...,we expect second quarter net asset value of <c...
5640,2013-04-29 06:20:00+00:00,False,[{'text': 'DNB’s underlying pre-tax profit imp...,1,Second best underlying pre-tax result in past ...,second best underlying pre tax result in past ...
23963,2014-04-16 16:10:00+00:00,False,[{'text': 'JM has delivered a declining number...,3,We expect a y-o-y growth in housing starts in ...,we expect a year over year growth in housing s...
6474,2014-02-18 08:21:00+00:00,False,[{'text': 'Indutrade acquires companies and cr...,1,Investment case – acquisition strategy now emb...,investment case – acquisition strategy now emb...
16193,2012-02-15 12:09:00+00:00,False,[{'text': 'We expected a good end to Q4 and at...,2,Higher staff costs offset by lower cost of goo...,higher staff costs offset by lower cost of goo...
7226,2014-11-25 07:18:00+00:00,False,[{'text': 'We re-initiate coverage of Pandora ...,1,Coverage re-initiated with Buy recommendation ...,coverage retained earnings initiated with buy ...
1425,2010-10-25 06:39:00+00:00,False,"[{'text': 'Volvo’s sales growth was 28%, in li...",1,Third quarter clean EBIT was 10% ahead of expe...,third quarter clean earnings before interest a...
7535,2015-04-23 06:30:00+00:00,False,[{'text': 'Following Nederman’s better than ex...,1,"Buy reiterated, target price up to SEK 250. Fo...",buy reiterated target price up to <currency> ...
6805,2014-07-15 05:09:00+00:00,False,[{'text': 'We expect global facilities service...,1,Expect soft Q2 organic growth and 10bp margin ...,expect soft second quarter organic growth and ...


In [22]:
df_reports['total_text'][8]

'Good start to the year but Q2 and Q3 the key for 2010 performance. The Q1 2010 revenues grew by 7.3% year-on-year and were in line with our estimate. The Q1 2010 EBIT of EUR 7.5m also showed growth, matching our EUR 7.6m forecast, with in line margins (consensus: EUR 4.6m). Tikkurila said that despite the good start to the year, the market remains challenging and visibility is not high at this time of the year. As Q1 2010 is a small quarter in terms of revenues and EBIT (Q2 and Q3 the key for the full year), the guidance for flat revenues and EBIT (ex. currency changes) was maintained despite the growth in Q1 2010. Estimates intact; we have 12.2% net sales and EUR 7m EBIT growth for 2010. SBU East already saw some volume growth in Q1 2010 and pre-order sales in Finland have started well, according to Tikkurila. Much of the full year performance is dependent on the exterior painting season but we believe our 2.8% volume growth estimate for 2010 is on the safe side. If the Swedish krona

In [3]:
df_indexed = df_reports[['total_text']]

In [4]:
df_indexed = df_reports[['total_text']]
df_indexed.index = range(1, len(df_indexed) + 1)  # set index from 1
index = df_indexed.index    
df_indexed.insert(0, 'index', index)
df_indexed = df_indexed[0:100]
print(df_indexed)

     index                                         total_text
1        1  Key focus on ramp-up of FBR and Singapore, and...
2        2  Pre-tax profit to balance around zero in March...
3        3  Pre-tax profits of SEK 116m in March – we expe...
4        4  Q1 EBITDA of NOK 415m, 43% above our estimate ...
5        5  We estimate Q1 revenue and EBIT growth on volu...
..     ...                                                ...
96      96  Top line good, management costs under control ...
97      97  Coverage terminated as the company is acquired...
98      98  Somewhat weaker volume but less FX impact. Vol...
99      99  Investment case remains intact after inclusion...
100    100  Issuance of Class C preference shares will inc...

[100 rows x 2 columns]


In [11]:
from transformers import RobertaForSequenceClassification, AlbertForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained(
    "/home/jupyter/pretrained-models/roberta-base/", 
    num_labels = 3,   
    output_attentions = False, 
    output_hidden_states = False, )
model.load_state_dict(torch.load("/home/jupyter/saved-models/roberta-base-500-finetuned.pth"))
model = model.roberta
tokenizer = RobertaTokenizer.from_pretrained('/home/jupyter/pretrained-models/roberta-base/')
model.eval()
context_embeddings = []
labels = []
with torch.no_grad():
    for record in df_indexed.to_dict('record'):
        ids = tokenizer.encode(record['total_text'])
        tokens = tokenizer.convert_ids_to_tokens(ids)
        #print(tokens)
        bert_output = model.forward(torch.tensor(ids).unsqueeze(0),encoder_hidden_states = True)
        final_layer_embeddings = bert_output[0][-1]
        #print(final_layer_embeddings)
        
        for i, token in enumerate(tokens):
            #print(f'{record["sn"]}_{token}', final_layer_embeddings[i])
            context_embeddings.append(final_layer_embeddings[i])
            labels.append(f'{record["index"]}_{token}')

In [12]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('summary/contextual_2_tensorboard_pt')

In [13]:
writer.add_embedding(torch.stack(context_embeddings),
                         metadata  = labels,
                        tag = f'contextual embeddings')